# No Glue Code

In [1]:
using Pkg
Pkg.activate("..")

  Activating project at `~/Cambdrige`


In [2]:
using Random
using LinearAlgebra
using PyPlot

#What we are tweaking
using Revise
using Turing
using MCMCChains

[ Info: Precompiling Turing [fce5fe82-541a-59a6-adf8-730c64b5f9a0]
┌ Warning: Module Tracker with build ID fafbfcfd-9efd-1a58-0000-064504379f44 is missing from the cache.
│ This may mean Tracker [9f7883ad-71c0-57eb-9f7f-b5c9e6d3789c] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1758
┌ Warning: Module Tracker with build ID fafbfcfd-9efd-1a58-0000-064504379f44 is missing from the cache.
│ This may mean Tracker [9f7883ad-71c0-57eb-9f7f-b5c9e6d3789c] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1758
┌ Warning: Module Tracker with build ID fafbfcfd-9efd-1a58-0000-064504379f44 is missing from the cache.
│ This may mean Tracker [9f7883ad-71c0-57eb-9f7f-b5c9e6d3789c] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1758
┌ Warning: Module Tracker with build ID fafbfcfd-9efd-1a58-0000-064504379f44 is missing from the cache.
│ This may mean Tracker [9f7883ad-71c0-5

## Model

In [3]:
# Just a simple Neal Funnel
d = 21
@model function funnel()
    θ ~ Normal(0, 3)
    z ~ MvNormal(zeros(d-1), exp(θ)*I)
    x ~ MvNormal(z, I)
end

funnel (generic function with 2 methods)

In [4]:
Random.seed!(1)
(;x) = rand(funnel() | (θ=0,))
funnel_model = funnel() | (;x)

DynamicPPL.Model{typeof(funnel), (), (), (), Tuple{}, Tuple{}, DynamicPPL.ConditionContext{NamedTuple{(:x,), Tuple{Vector{Float64}}}, DynamicPPL.DefaultContext}}(funnel, NamedTuple(), NamedTuple(), ConditionContext((x = [1.2142074831535152, 1.23371919965455, -0.8480146960461767, 0.1600994648479841, 1.9180385508479283, -3.401523464506408, -0.0957684186471088, 0.6734622629464286, -3.2749467689509633, -1.6760091758453226, 1.9567202902549736, 0.1136169088905351, 0.11117896909388916, -0.5373922347882832, -0.12436857036298687, -1.2901071061088532, 1.702584517514787, -0.44460133117954226, 1.0818722439221686, 1.2208011493237483],), DynamicPPL.DefaultContext()))

## Sampling

In [5]:
nadapts=500 
TAP=0.95
nuts = Turing.NUTS(nadapts, TAP; init_ϵ=0.1)

NUTS{Turing.Essential.ForwardDiffAD{0}, (), AdvancedHMC.DiagEuclideanMetric}(500, 0.95, 10, 1000.0, 0.1)

In [6]:
nuts_samples = sample(funnel_model, nuts, 5000)

LoadError: MethodError: sample(::TaskLocalRNG, ::DynamicPPL.Model{typeof(funnel), (), (), (), Tuple{}, Tuple{}, DynamicPPL.ConditionContext{NamedTuple{(:x,), Tuple{Vector{Float64}}}, DynamicPPL.DefaultContext}}, ::DynamicPPL.Sampler{NUTS{Turing.Essential.ForwardDiffAD{0}, (), AdvancedHMC.DiagEuclideanMetric}}, ::Int64) is ambiguous.

Candidates:
  sample([90mrng[39m::[1mAbstractRNG[22m, [90mmodel[39m::[1mAbstractMCMC.AbstractModel[22m, [90msampler[39m::[1mDynamicPPL.Sampler[22m[0m{<:Turing.Inference.AdaptiveHamiltonian}, [90mN[39m::[1mInteger[22m; chain_type, resume_from, progress, nadapts, discard_adapt, discard_initial, kwargs...)
[90m    @[39m [90mTuring.Inference[39m [90m~/Cambdrige/Turing.jl/src/inference/[39m[90m[4mhmc.jl:103[24m[39m
  sample([90mrng[39m::[1mAbstractRNG[22m, [90mmodel[39m::[1mAbstractMCMC.AbstractModel[22m, [90msampler[39m::[1mDynamicPPL.Sampler[22m[0m{<:Turing.Inference.InferenceAlgorithm}, [90mN[39m::[1mInteger[22m; c

In [ ]:
spl = Turing.Sampler(Turing.NUTS(), funnel_model)

DynamicPPL.Sampler{NUTS{Turing.Essential.ForwardDiffAD{0}, (), AdvancedHMC.DiagEuclideanMetric}}(NUTS{Turing.Essential.ForwardDiffAD{0}, (), AdvancedHMC.DiagEuclideanMetric}(-1, 0.65, 10, 1000.0, 0.0), DynamicPPL.Selector(0x000019f8183c3710, :default, false))

In [ ]:
spl.model

LoadError: type Sampler has no field model